In [1]:
#!pip -q install einops
import numpy as np
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from set_data import load_real_weather, WeatherPairs, split_windows_train_val_test
from visualization import (plot_complete_data, 
                           plot_weatherpair_panel,
                           visualize_loss, plot_ctx_and_scenarios_panel, 
                           plot_ctx_and_scenarios_panel_ci)
from utils import *
from diffusion_model import DiffusionSchedule, TinyCondUNet1D
from diffusion_model_train import train_one_epoch, eval_epoch, EarlyStopping
from diffusion_model_generate import p_sample_loop
import error_metrics as em
np.set_printoptions(suppress=True)

VARS = 6
EPOCHS = 100
feature_names = ["T2M_MIN", "T2M_MAX", "PRECTOTCORR", "ALLSKY_SFC_SW_DWN", "RH2M", "WS2M"]

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)



In [2]:

X, columns = load_real_weather("../../data/data_19950630_to_20250630.csv", 
                      start_date="1995-06-30", end_date="2024-06-30")
print("Real data shape:", X.shape)  # (days, number of features)


In [3]:
plot_complete_data(X, columns)

In [4]:
CTX = 90
HORIZON = 90
dataset = WeatherPairs(X, ctx=CTX, horizon=HORIZON, stride=1)
train_ds, val_ds, test_ds = split_windows_train_val_test(dataset, test_frac=0.1, val_frac=0.1, train_frac=0.8)



In [5]:
plot_weatherpair_panel(train_ds, i=0, feature_names=feature_names)

In [6]:
print("windows:", len(dataset))
print("train/val/test:", len(train_ds), len(val_ds), len(test_ds))
print("train date: ", train_ds.dataset.data[train_ds.indices[0]])
print("test date: ", test_ds.dataset.data[test_ds.indices[0]])
print("val date: ", val_ds.dataset.data[val_ds.indices[0]])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False, drop_last=False)


In [7]:
mean, std = fit_standardizer(train_ds)
mean_t = torch.tensor(mean, device=device)
std_t = torch.tensor(std, device=device)
print("mean:", mean, "\nstd:", std)

In [8]:
sched = DiffusionSchedule(T=200)

In [9]:
model = TinyCondUNet1D(in_vars=VARS, ctx_vars=VARS).to(device)


In [ ]:
#opt = torch.optim.RMSprop(model.parameters(), lr=2e-4)
opt = torch.optim.SGD(
    model.parameters(),
    lr=1e-2,
    momentum=0.9,
    weight_decay=1e-4,
    nesterov=True
)

train_hist, val_hist = [], []

early = EarlyStopping(patience=20, min_delta=1e-4, mode="min")

for e in range(1, EPOCHS + 1):
    tr = train_one_epoch(opt, model, sched, device, train_loader, mean_t, std_t)
    va = eval_epoch(model, device, val_loader, sched, mean_t, std_t)

    train_hist.append(float(tr))
    val_hist.append(float(va))

    print(f"epoch {e:02d} | train {tr:.4f} | val {va:.4f} | best {early.best:.4f} (bad {early.num_bad}/{early.patience})")

    if early.step(va, model, e):
        print(f"Early stopping at epoch {e:02d}. Best val={early.best:.4f} at epoch {early.best_epoch:02d}.")
        break

# restore best weights before final eval / saving
early.restore_best(model)

visualize_loss(len(train_hist), train_hist, val_hist)


In [ ]:
df_dates = pd.read_csv('dates_loaded.csv', index_col=0)
index_date = df_dates[df_dates.DATE=='2023-09-01'].index.values[0]
print(index_date)

index_test = np.where(test_ds.indices== index_date)[0][0]

In [ ]:
ctx_raw, tgt_raw = test_ds[index_test]
ctx_raw = ctx_raw.unsqueeze(0)  # (1,60,6)
tgt_raw = tgt_raw.numpy()       # (60,6)
pd.DataFrame(ctx_raw.squeeze(0).numpy(), columns=feature_names).to_csv('fixed_window.csv')
scens = p_sample_loop(ctx_raw, model, device, sched, HORIZON, VARS, 
                      mean_t, std_t, n_scenarios=500).numpy() 
days = np.arange(HORIZON)


In [ ]:
plot_ctx_and_scenarios_panel(
    ctx_raw,
    tgt_raw,
    scens,
    feature_names=feature_names,
    max_scen_to_plot=100
)


In [ ]:
plot_ctx_and_scenarios_panel_ci(ctx_raw, tgt_raw, scens)

In [ ]:
np.save("scenarios.npy", scens)

In [ ]:
Omega = np.load("scenarios.npy")

In [ ]:
df_summary = pd.DataFrame()

for i in range(test_ds.__len__()):
    ctx_raw, tgt_raw = test_ds[i]
    tgt_raw = tgt_raw.detach().cpu().numpy()
    ctx_raw = ctx_raw.unsqueeze(0)  # (1,CTX,VARS)
    scens = p_sample_loop(ctx_raw, model, device, sched, HORIZON, VARS,
                      mean_t, std_t, n_scenarios=50).detach().cpu().numpy()
    
    mse  = em.mse_ensemble_mean_batch(scens, tgt_raw).reshape(-1, 1)
    mae = em.mae_ensemble_mean_batch(scens, tgt_raw).reshape(-1, 1)
    crps_nd = em.crps_ensemble_snd(scens, tgt_raw, average=False).mean(axis=0)  # (60, 6)
    #print(f"Test sample {i+1}/{test_ds.__len__()} | MSE: {mse.mean():.4f} | MAE: {mae.mean():.4f} | CRPS_ND: {crps_nd.mean():.4f}")
    
    df_temp = pd.DataFrame({
        "mse": mse.flatten(),   
        "mae": mae.flatten(),
        "crps_nd": crps_nd.flatten()
    })
    df_summary = pd.concat([df_summary, df_temp], axis=0)


df_summary


In [ ]:
# pick a test sample
# i = 0
# ctx, tgt = test_ds[i]     # torch tensors

# move to numpy
# y = tgt.detach().cpu().numpy()   # shape: (H, D)
# inverse transform (broadcasting works)

In [ ]:
# y

In [ ]:
# Omega.shape

In [ ]:

# mse  = em.mse_ensemble_mean_batch(Omega, y)
# mae = em.mae_ensemble_mean_batch(Omega, y)

#print(f"MSE  : {mse:.4f}")
#print(f"MAPE : {mape:.2f}%")
#print(f"CRPS : {crps:.4f}")
# mse


In [ ]:
# mae

In [ ]:
# var_names = ["T2M_MIN", "T2M_MAX", "PRECTOTCORR", "ALLSKY_SFC_SW_DWN", "RH2M", "WS2M"]
# crps_nd = em.crps_ensemble_snd(Omega, y, average=False)  # (60, 6)
# crps_per_var = crps_nd.mean(axis=0)  # (6,)
# for name, val in zip(var_names, crps_per_var):
#    print(name, val)


In [ ]:
#y.shape

In [ ]:

var_names = ["tmin", "tmax", "precip", "wind", "humidity", "solar"]

#X_mean = Omega.mean(axis=0)   # (60, 6)
X_mean = Omega[0]   # (60, 6)
df = pd.DataFrame(X_mean, columns=var_names)

sns.pairplot(df, corner=True, diag_kind="hist")
plt.show()

In [ ]:
# y.shape

In [ ]:
#X_mean = Omega.mean(axis=0)   # (60, 6)
i = 0
ctx, tgt = test_ds[i]     # torch tensors

# move to numpy
X_mean = tgt.detach().cpu().numpy()   # shape: (H, D)

# X_mean = y   # (60, 6)
df = pd.DataFrame(X_mean, columns=var_names)

sns.pairplot(df, corner=True, diag_kind="hist")
plt.show()

In [ ]:
# Omega.shape

In [ ]:
new_df = pd.DataFrame()
for i in range(Omega.shape[0]):
     new_df = pd.concat([new_df, pd.DataFrame(Omega[i], columns=var_names)], axis=0)

sns.pairplot(new_df, corner=True, diag_kind="hist")
plt.show()


In [ ]:
sns.pairplot(pd.DataFrame(X), corner=True, diag_kind="hist")
plt.show()

In [ ]:
new_df.corr()

In [ ]:
df.corr()